In [1]:
import numpy as np
import pandas as pd
import pickle

import matplotlib.pyplot as plt
import seaborn as sb


from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score



import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.preprocessing import image
from keras.wrappers.scikit_learn import KerasClassifier

## Loading the dataframe
Loading the dataframe from the pickle file which was exported from Pre processing stage

In [2]:
with open("/Volumes/Macintosh HD - Data/Users/admin/Documents/HD Drive/DataProjects/EnergyInnovations-ML/Data/df_cleaned.pkl",'rb') as file:
    df = pickle.load(file)

df

,Name,GFA,BType,Energy,WeekDay,Hour,TAVG,TMIN,TMAX,Sem
0,Piper Writing Center,4314.0,Academic,1.4,1,00:00:00,43.0,38,51,0
1,Piper Writing Center,4314.0,Academic,2.6,1,01:00:00,43.0,38,51,0
2,Piper Writing Center,4314.0,Academic,0.8,1,02:00:00,43.0,38,51,0
3,Piper Writing Center,4314.0,Academic,0.8,1,03:00:00,43.0,38,51,0
4,Piper Writing Center,4314.0,Academic,5.0,1,04:00:00,43.0,38,51,0
...,...,...,...,...,...,...,...,...,...,...
602573,San Pablo,77484.0,Housing,51.6,1,19:00:00,54.0,45,59,0
602574,San Pablo,77484.0,Housing,52.4,1,20:00:00,54.0,45,59,0
602575,San Pablo,77484.0,Housing,50.5,1,21:00:00,54.0,45,59,0
602576,San Pablo,77484.0,Housing,51.5,1,22:00:00,54.0,45,59,0


# Data Understanding and Transformation

In [133]:
df.dtypes

Name        object
GFA        float64
BType       object
Energy      object
WeekDay      int64
Hour        object
TAVG       float64
TMIN         int64
TMAX         int64
Sem          int64
dtype: object

Converting Energy datatype from string to float

In [3]:
df['Energy'] = df['Energy'].apply(lambda x:float(x)).round(1)

#### Checking for negative and missing values for energy

In [4]:
df.loc[df['Energy']<0,['Energy']].count()

Energy    0
dtype: int64

In [5]:
df.loc[df['Energy']<0,['Name']].nunique()

Name    0
dtype: int64

In [6]:
df.isna().sum()

Name       0
GFA        0
BType      0
Energy     0
WeekDay    0
Hour       0
TAVG       0
TMIN       0
TMAX       0
Sem        0
dtype: int64

#### Removing the negative and missing values

In [7]:
df = df[df['Energy']>0].dropna()

In [8]:
df['Energy'].describe()

count    571373.000000
mean        109.248414
std         121.975879
min           0.100000
25%          34.900000
50%          66.100000
75%         137.100000
max         757.300000
Name: Energy, dtype: float64

In [9]:
df['Energy'].apply(lambda x:x)

0          1.4
1          2.6
2          0.8
3          0.8
4          5.0
          ... 
602573    51.6
602574    52.4
602575    50.5
602576    51.5
602577    49.7
Name: Energy, Length: 571373, dtype: float64

In [ ]:
'''
with open('Data/df_cleaned.pkl','wb') as file:
    pickle.dump(df,file)
'''

In [11]:
df

,Name,GFA,BType,Energy,WeekDay,Hour,TAVG,TMIN,TMAX,Sem
0,Piper Writing Center,4314.0,Academic,1.4,1,00:00:00,43.0,38,51,0
1,Piper Writing Center,4314.0,Academic,2.6,1,01:00:00,43.0,38,51,0
2,Piper Writing Center,4314.0,Academic,0.8,1,02:00:00,43.0,38,51,0
3,Piper Writing Center,4314.0,Academic,0.8,1,03:00:00,43.0,38,51,0
4,Piper Writing Center,4314.0,Academic,5.0,1,04:00:00,43.0,38,51,0
...,...,...,...,...,...,...,...,...,...,...
602573,San Pablo,77484.0,Housing,51.6,1,19:00:00,54.0,45,59,0
602574,San Pablo,77484.0,Housing,52.4,1,20:00:00,54.0,45,59,0
602575,San Pablo,77484.0,Housing,50.5,1,21:00:00,54.0,45,59,0
602576,San Pablo,77484.0,Housing,51.5,1,22:00:00,54.0,45,59,0


# One hot encoding Categorical

In [12]:
BTypeDF = pd.get_dummies(df['BType'],prefix='BType')
HourDF = pd.get_dummies(df['Hour'],prefix='Hour')
df = pd.concat([df,BTypeDF,HourDF],axis =1)
df

,Name,GFA,BType,Energy,WeekDay,Hour,TAVG,TMIN,TMAX,Sem,...,Hour_14:00:00,Hour_15:00:00,Hour_16:00:00,Hour_17:00:00,Hour_18:00:00,Hour_19:00:00,Hour_20:00:00,Hour_21:00:00,Hour_22:00:00,Hour_23:00:00
0,Piper Writing Center,4314.0,Academic,1.4,1,00:00:00,43.0,38,51,0,...,0,0,0,0,0,0,0,0,0,0
1,Piper Writing Center,4314.0,Academic,2.6,1,01:00:00,43.0,38,51,0,...,0,0,0,0,0,0,0,0,0,0
2,Piper Writing Center,4314.0,Academic,0.8,1,02:00:00,43.0,38,51,0,...,0,0,0,0,0,0,0,0,0,0
3,Piper Writing Center,4314.0,Academic,0.8,1,03:00:00,43.0,38,51,0,...,0,0,0,0,0,0,0,0,0,0
4,Piper Writing Center,4314.0,Academic,5.0,1,04:00:00,43.0,38,51,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
602573,San Pablo,77484.0,Housing,51.6,1,19:00:00,54.0,45,59,0,...,0,0,0,0,0,1,0,0,0,0
602574,San Pablo,77484.0,Housing,52.4,1,20:00:00,54.0,45,59,0,...,0,0,0,0,0,0,1,0,0,0
602575,San Pablo,77484.0,Housing,50.5,1,21:00:00,54.0,45,59,0,...,0,0,0,0,0,0,0,1,0,0
602576,San Pablo,77484.0,Housing,51.5,1,22:00:00,54.0,45,59,0,...,0,0,0,0,0,0,0,0,1,0


# Prediction Modeling

## Simple Linear Regression

In [9]:
values = df.drop(['TMIN','TMAX','Name','Hour'],axis=1).values

#scaling

scaler = MinMaxScaler(feature_range = (0,1))
ScaledValues = pd.DataFrame(scaler.fit_transform(values))
X = ScaledValues

In [ ]:
#ran the model on EC2 instance for performance

LinReg = LinearRegression(normalize = True)
LinReg.fit(X,y)
score  = LinReg.score(X,y)

In [146]:
file = open("Data/linear.pkl",'rb')
score = pickle.load(file)

score

0.1330667617808884

## Neural Networks

In [13]:
featureData = df.drop(['Energy','Name','BType','Hour'],axis=1).values

#scaling

scaler = MinMaxScaler(feature_range = (0,1))
X = scaler.fit_transform(featureData)

In [14]:
X.shape

(571373, 34)

In [15]:
Energy = df['Energy'].values

bins = np.linspace(0,1000,20)
Energy = pd.cut(Energy,bins,labels = False)

In [16]:
EnergyDF =pd.DataFrame(Energy)
EnergyDF.nunique()

0    15
dtype: int64

In [17]:
y = pd.get_dummies(Energy,prefix='Energy')
y

,Energy_0,Energy_1,Energy_2,Energy_3,Energy_4,Energy_5,Energy_6,Energy_7,Energy_8,Energy_9,Energy_10,Energy_11,Energy_12,Energy_13,Energy_14
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
571368,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
571369,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
571370,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
571371,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### Initializing and parameter tuning

In [25]:
model3 = Sequential()
model3.add(Dense(200,input_dim=34,activation='relu', name = 'layer_1'))
model3.add(Dense(130,activation='relu', name = 'layer_2'))
model3.add(Dropout(0.5))
model3.add(Dense(80,activation='relu',name = 'layer_3'))
model3.add(Dense(80,activation='relu',name = 'layer_4'))
model3.add(Dense(15,activation='softmax',name = 'output_layer'))

model3.compile(loss='categorical_crossentropy', optimizer='SGD', metrics=['accuracy'])

In [26]:
model3.fit(X,y,epochs=150,shuffle=True,verbose=2)

error_rate,accuracy = model3.evaluate(X,y,verbose=0)

Epoch 1/150
17856/17856 - 19s - loss: 1.3935 - accuracy: 0.4962
Epoch 2/150
17856/17856 - 28s - loss: 1.2084 - accuracy: 0.5577
Epoch 3/150
17856/17856 - 21s - loss: 1.1499 - accuracy: 0.5745
Epoch 4/150
17856/17856 - 20s - loss: 1.1117 - accuracy: 0.5852
Epoch 5/150
17856/17856 - 19s - loss: 1.0815 - accuracy: 0.5938
Epoch 6/150
17856/17856 - 21s - loss: 1.0545 - accuracy: 0.6025
Epoch 7/150
17856/17856 - 20s - loss: 1.0323 - accuracy: 0.6080
Epoch 8/150
17856/17856 - 19s - loss: 1.0126 - accuracy: 0.6114
Epoch 9/150
17856/17856 - 20s - loss: 0.9956 - accuracy: 0.6150
Epoch 10/150
17856/17856 - 21s - loss: 0.9792 - accuracy: 0.6189
Epoch 11/150
17856/17856 - 22s - loss: 0.9668 - accuracy: 0.6210
Epoch 12/150
17856/17856 - 21s - loss: 0.9530 - accuracy: 0.6232
Epoch 13/150
17856/17856 - 20s - loss: 0.9413 - accuracy: 0.6262
Epoch 14/150
17856/17856 - 20s - loss: 0.9312 - accuracy: 0.6289
Epoch 15/150
17856/17856 - 19s - loss: 0.9219 - accuracy: 0.6302
Epoch 16/150
17856/17856 - 20s - l

Epoch 127/150
17856/17856 - 28s - loss: 0.7300 - accuracy: 0.6876
Epoch 128/150
17856/17856 - 26s - loss: 0.7292 - accuracy: 0.6878
Epoch 129/150
17856/17856 - 24s - loss: 0.7275 - accuracy: 0.6881
Epoch 130/150
17856/17856 - 23s - loss: 0.7285 - accuracy: 0.6882
Epoch 131/150
17856/17856 - 24s - loss: 0.7277 - accuracy: 0.6881
Epoch 132/150
17856/17856 - 23s - loss: 0.7258 - accuracy: 0.6892
Epoch 133/150
17856/17856 - 24s - loss: 0.7266 - accuracy: 0.6889
Epoch 134/150
17856/17856 - 24s - loss: 0.7252 - accuracy: 0.6897
Epoch 135/150
17856/17856 - 23s - loss: 0.7267 - accuracy: 0.6887
Epoch 136/150
17856/17856 - 23s - loss: 0.7262 - accuracy: 0.6891
Epoch 137/150
17856/17856 - 25s - loss: 0.7246 - accuracy: 0.6891
Epoch 138/150
17856/17856 - 23s - loss: 0.7245 - accuracy: 0.6890
Epoch 139/150
17856/17856 - 23s - loss: 0.7240 - accuracy: 0.6895
Epoch 140/150
17856/17856 - 22s - loss: 0.7227 - accuracy: 0.6901
Epoch 141/150
17856/17856 - 23s - loss: 0.7238 - accuracy: 0.6895
Epoch 142/

### Evaluating with K fold Cross validation

In [ ]:
#Building the model and evaluating using K fold cross validation
'''
RunName = 'model4'
def createmodel():
    model = Sequential()
    model.add(Dense(10,input_dim=59,activation='relu'))
    modle.add(Dense(5,activation='relu'))
    model.add(Dense(3,activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

seed = 7
np.random.seed(seed)

model = KerasClassifier(build_fn = createmodel,epochs = 50, batch_size = 10, verbose = 0)
kfold = KFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(model, X, y, cv=kfold)
print(results.mean())'''